# **Method 1: K Means Clustering**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

In [ ]:
df = pd.read_csv('input_game.csv')

In [ ]:
df['p1_action'] = df['p1_action'].replace({'TRUST': 1, 'CHEAT': 0})
df['p2_action'] = df['p2_action'].replace({'TRUST': 1, 'CHEAT': 0})
df.head()

,game_id,p1_id,p2_id,p1_action,p2_action,turn
0,0,174,125,1,0,1
1,0,174,125,0,1,2
2,0,174,125,1,1,3
3,0,174,125,1,1,4
4,0,174,125,1,1,5


In [ ]:
# Function to calculate moving average
def calculate_moving_average(actions, alpha):
    weighted_avg = []
    actions = actions.reset_index()
    for i in range(len(actions)):
        total_weighted_sum = 0
        sum_of_weights = 0
        for j in range(i + 1):
            total_weighted_sum += actions.iloc[i - j, 1] * (alpha ** j)
            sum_of_weights += alpha ** j
        weighted_avg.append(total_weighted_sum / sum_of_weights)
    return weighted_avg


def player_tp(player_id, alpha=0.9):
    # Assuming alpha value for the moving average

    # Player ID you're interested in (p1_id or p2_id)

    # Selecting games where the player has played
    selected_games = df[(df['p1_id'] == player_id) | (df['p2_id'] == player_id)]

    # Resetting index for easy iteration
    selected_games = selected_games.reset_index(drop=True)

    # Plotting
    # fig, axs = plt.subplots()

    # Dictionary to store colors for each game_id
    color_dict = {}
    x_coords = []
    y_coords = []
    for idx, game_id in enumerate(selected_games['game_id'].unique()):
        game_data = selected_games[selected_games['game_id'] == game_id]
        game_data = game_data.reset_index()
        if game_data['p2_id'].iloc[0] == player_id:
            # Calculate moving average for p1's actions
            game_data['MA1'] = calculate_moving_average(game_data['p1_action'], alpha)
            # Shift p2's actions
            df_shifted_index = game_data['p2_action'].shift(-1)
            # axs.scatter(game_data['MA1'], df_shifted_index, label=f'Game {game_id}', color=f'C{idx}')
            x_coords += game_data['MA1'].tolist()
            y_coords += df_shifted_index.tolist()
        else:
            # Calculate moving average for p2's actions
            game_data['MA2'] = calculate_moving_average(game_data['p2_action'], alpha)
            # Shift p1's actions
            df_shifted_index = game_data['p1_action'].shift(-1)
            # axs.scatter(game_data['MA2'], df_shifted_index, label=f'Game {game_id}', color=f'C{idx}')
            x_coords += game_data['MA2'].tolist()
            y_coords += df_shifted_index.tolist()

        color_dict[game_id] = f'C{idx}'

    # axs.set_xlabel('MA')
    # axs.set_ylabel('p_action (Shifted Index)')
    # axs.set_title('MA vs. Shifted p_action')

    # Adding legend
    # axs.legend()
    p = np.array(x_coords[:-1])
    q = np.array(y_coords[:-1])
    mask = ~np.isnan(p) & ~np.isnan(q)
    da = np.polyfit(p[mask],q[mask],3)
    tp = np.poly1d(da)


    # axs.plot(p[mask],tp(p[mask]))

    # df_plot = pd.DataFrame({'MA': x_coords, 'p_action': y_coords})
    # sns.catplot(kind='violin', data=df_plot, hue="p_action", x="MA", split=True)
    # plt.show()
    return tp

In [ ]:
df_tot = pd.concat([df['p1_id'], df['p2_id']])
yx = [0 for _ in range(201)]

for i in range(1,202):
  tpp09 = player_tp(i)
  tpp01 = player_tp(i, 0.1)
  tpp1 = player_tp(i, 1)
  tpp0 = player_tp(i, 0)
  x = []
  for j in range(0,105,5):
    x.append(j/100)
  x = np.array(x)
  y0 = tpp0(x)
  y01 = tpp01(x)
  y09 = tpp09(x)
  y1 = tpp1(x)
  yx[i-1] = y0 + y01 + y09 + y1

<ipython-input-5-870455ddfa60>:64: RankWarning: Polyfit may be poorly conditioned
  da = np.polyfit(p[mask],q[mask],3)
<ipython-input-5-870455ddfa60>:64: RankWarning: Polyfit may be poorly conditioned
  da = np.polyfit(p[mask],q[mask],3)
<ipython-input-5-870455ddfa60>:64: RankWarning: Polyfit may be poorly conditioned
  da = np.polyfit(p[mask],q[mask],3)
<ipython-input-5-870455ddfa60>:64: RankWarning: Polyfit may be poorly conditioned
  da = np.polyfit(p[mask],q[mask],3)
<ipython-input-5-870455ddfa60>:64: RankWarning: Polyfit may be poorly conditioned
  da = np.polyfit(p[mask],q[mask],3)
<ipython-input-5-870455ddfa60>:64: RankWarning: Polyfit may be poorly conditioned
  da = np.polyfit(p[mask],q[mask],3)
<ipython-input-5-870455ddfa60>:64: RankWarning: Polyfit may be poorly conditioned
  da = np.polyfit(p[mask],q[mask],3)
<ipython-input-5-870455ddfa60>:64: RankWarning: Polyfit may be poorly conditioned
  da = np.polyfit(p[mask],q[mask],3)
<ipython-input-5-870455ddfa60>:64: RankWarning: 

In [ ]:
from sklearn.metrics import silhouette_score

# Assuming strategies is a list of lists where each inner list represents the strategy encoding of a player
# Convert strategies into a numpy array
strategies_array = np.array(yx)

# Define a range of cluster numbers to evaluate
min_clusters = 2
max_clusters = 12

best_score = -1
best_num_clusters = None

# Iterate over different numbers of clusters and evaluate silhouette score
for num_clusters in range(min_clusters, max_clusters + 1):
    kmeans = KMeans(n_clusters=num_clusters)
    clusters = kmeans.fit_predict(strategies_array)
    silhouette_avg = silhouette_score(strategies_array, clusters)
    print(f"For {num_clusters} clusters, the silhouette score is {silhouette_avg}")

    # Update the best score and number of clusters if needed
    if silhouette_avg > best_score:
        best_score = silhouette_avg
        best_num_clusters = num_clusters

# Print the best number of clusters
print(f"The best number of clusters is: {best_num_clusters}")


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

For 2 clusters, the silhouette score is 0.5147455764933634
For 3 clusters, the silhouette score is 0.5210664201852596
For 4 clusters, the silhouette score is 0.4953003863228335
For 5 clusters, the silhouette score is 0.5321729703324583
For 6 clusters, the silhouette score is 0.5513941660086282


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


For 7 clusters, the silhouette score is 0.5644230519794273
For 8 clusters, the silhouette score is 0.5700843841823086
For 9 clusters, the silhouette score is 0.58597831395248
For 10 clusters, the silhouette score is 0.6079650354222967
For 11 clusters, the silhouette score is 0.6363771891784176
For 12 clusters, the silhouette score is 0.6079763902886309
The best number of clusters is: 11


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
# Assuming strategies is a list of lists where each inner list represents the strategy encoding of a player
# Convert strategies into a numpy array
strategies_array = np.array(yx)

# Number of clusters (you may adjust this based on your requirements)
num_clusters = 11

# Perform K-means clustering
kmeans = KMeans(n_clusters=num_clusters)
clusters = kmeans.fit_predict(strategies_array)

# Print the cluster assignments
f = [[] for _ in range(11)]
for i, cluster in enumerate(clusters):
  f[cluster].append(i)
print(f)

[[0, 4, 9, 10, 16, 25, 27, 33, 34, 39, 40, 43, 45, 46, 47, 48, 50, 51, 53, 55, 57, 62, 65, 69, 72, 74, 81, 87, 88, 89, 91, 97, 101, 105, 106, 111, 119, 122, 124, 129, 133, 134, 143, 146, 150, 151, 152, 156, 162, 163, 164, 167, 168, 169, 176, 181, 182, 183, 184, 187, 194, 197, 200], [13, 21, 22, 35, 36, 37, 52, 54, 56, 63, 64, 95, 96, 104, 130, 140, 145, 147, 166, 188, 196, 199], [5, 6, 7, 30, 42, 60, 98, 126, 132, 138, 185, 190, 195], [29, 41, 66, 76, 78, 92, 112, 158, 173, 178, 179], [15, 17, 71, 77, 80, 93, 115, 121, 141, 144, 159, 165], [1, 2, 19, 99, 117, 125, 137, 148, 149, 189], [8, 12, 59, 94, 103, 120, 127, 136, 174, 177, 180, 191, 192, 193], [11, 18, 26, 38, 44, 68, 73, 75, 79, 85, 86, 100, 102, 142, 153, 160, 161, 170, 175, 198], [20, 28, 58, 61, 83, 90, 108, 116, 131, 139], [3, 14, 24, 32, 84, 107, 109, 110, 114, 128, 135, 154, 157, 171], [23, 31, 49, 67, 70, 82, 113, 118, 123, 155, 172, 186]]


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


# **Method 2: Finding strategies manually**

In [ ]:
df = pd.read_csv('input_game.csv')
df['p1_action'] = df['p1_action'].replace({'TRUST': 1, 'CHEAT': 0})
df['p2_action'] = df['p2_action'].replace({'TRUST': 1, 'CHEAT': 0})
df.head()

,game_id,p1_id,p2_id,p1_action,p2_action,turn
0,0,174,125,1,0,1
1,0,174,125,0,1,2
2,0,174,125,1,1,3
3,0,174,125,1,1,4
4,0,174,125,1,1,5


In [ ]:
players = {k:-1 for k in range(1,202)}

In [ ]:
def list_strategy(stratid):
  playerss = []
  for i in players:
    if players[i] == stratid:
      playerss.append(i)
  return playerss

def check_strategy(check, stratid):
  eliminated = []
  for i in players:
    if players[i]==-1 and check(i):
      players[i] = stratid
      eliminated.append(i)
  return eliminated

def unclassified_players():
  unclassified = []
  for i in players:
    if players[i] == -1:
      unclassified.append(i)
  return unclassified

def analyse_player(player_id):
  selected_games = df[(df['p1_id'] == player_id) | (df['p2_id'] == player_id)]

  # Resetting index for easy iteration
  selected_games = selected_games.reset_index(drop=True)['game_id'].unique()
  return selected_games

In [ ]:
# strategy 1, check for 90% 1's
def check1(player_id):
  cutoff = 0.9
  # Selecting games where the player has played
  selected_games = df[(df['p1_id'] == player_id) | (df['p2_id'] == player_id)]
  selected_games = selected_games.reset_index(drop=True)
  a = []
  for idx, game_id in enumerate(selected_games['game_id'].unique()):
    game_data = selected_games[selected_games['game_id'] == game_id]
    game_data = game_data.reset_index()
    if game_data['p1_id'].iloc[0] == player_id:
      a += game_data['p1_action'].tolist()
    elif game_data['p2_id'].iloc[0] == player_id:
      a += game_data['p2_action'].tolist()
  return np.array(a).mean() >= cutoff

In [ ]:
# strategy 2, check for 90% 0's
def check2(player_id):
  cutoff = 0.1
  # Selecting games where the player has played
  selected_games = df[(df['p1_id'] == player_id) | (df['p2_id'] == player_id)]
  selected_games = selected_games.reset_index(drop=True)
  a = []
  for idx, game_id in enumerate(selected_games['game_id'].unique()):
    game_data = selected_games[selected_games['game_id'] == game_id]
    game_data = game_data.reset_index()
    if game_data['p1_id'].iloc[0] == player_id:
      a += game_data['p1_action'].tolist()
    elif game_data['p2_id'].iloc[0] == player_id:
      a += game_data['p2_action'].tolist()
  return np.array(a).mean() <= cutoff

In [ ]:
# strategy 3, check for 90% tit-for-tat
def check3(player_id):
  cutoff = 0.9
  # Selecting games where the player has played
  selected_games = df[(df['p1_id'] == player_id) | (df['p2_id'] == player_id)]
  selected_games = selected_games.reset_index(drop=True)
  a = []
  for idx, game_id in enumerate(selected_games['game_id'].unique()):
    game_data = selected_games[selected_games['game_id'] == game_id]
    game_data = game_data.reset_index()
    for i, game in enumerate(game_data):
      if i == 0:
        continue
      if game_data['p1_id'].iloc[0] == player_id:
        if game_data['p2_action'].iloc[i-1] == game_data['p1_action'].iloc[i]:
          a.append(1)
        else:
          a.append(0)
      elif game_data['p2_id'].iloc[0] == player_id:
        if game_data['p1_action'].iloc[i-1] == game_data['p2_action'].iloc[i]:
          a.append(1)
        else:
          a.append(0)
  return np.array(a).mean() >= cutoff

In [ ]:
# strategy 4, check for 90% spiteful
def check4(player_id):
  cutoff = 0.9
  # Selecting games where the player has played
  selected_games = df[(df['p1_id'] == player_id) | (df['p2_id'] == player_id)]
  selected_games = selected_games.reset_index(drop=True)
  a = []
  for idx, game_id in enumerate(selected_games['game_id'].unique()):
    game_data = selected_games[selected_games['game_id'] == game_id]
    game_data = game_data.reset_index()
    pos, opp = 2, 1
    if game_data['p1_id'].iloc[0] == player_id:
      pos = 1
      opp = 2
    alldef = False
    for i, game in game_data.iterrows():
      if not alldef:
        if game[f'p{pos}_action'] == 1:
          a.append(1)
        else:
          a.append(0)
      else:
        if game[f'p{pos}_action'] == 0:
          a.append(1)
        else:
          a.append(0)
      if game[f'p{opp}_action'] == 0:
        alldef = True
  return np.array(a).mean() >= cutoff

In [ ]:
# strategy 5, check for 90% soft_mojo
def check5(player_id):
  cutoff = 0.8
  # Selecting games where the player has played
  selected_games = df[(df['p1_id'] == player_id) | (df['p2_id'] == player_id)]
  selected_games = selected_games.reset_index(drop=True)
  a = []
  for idx, game_id in enumerate(selected_games['game_id'].unique()):
    game_data = selected_games[selected_games['game_id'] == game_id]
    game_data = game_data.reset_index()
    pos, opp = 2, 1
    if game_data['p1_id'].iloc[0] == player_id:
      pos = 1
      opp = 2
    defs = 0
    coops = 0
    for i, game in game_data.iterrows():
      if coops >= defs:
        if game[f'p{pos}_action'] == 1:
          a.append(1)
        else:
          a.append(0)
      else:
        if game[f'p{pos}_action'] == 1:
          a.append(0)
        else:
          a.append(1)
      if game[f'p{opp}_action'] == 0:
        defs += 1
      else:
        coops += 1
  return np.array(a).mean() >= cutoff

In [ ]:
check_strategy(check1,1)
print(len(list_strategy(1)))
list_strategy(1)

14


[9, 13, 60, 95, 104, 121, 128, 137, 175, 178, 181, 192, 193, 194]

In [ ]:
check_strategy(check2,2)
print(len(list_strategy(2)))
list_strategy(2)

13


[4, 15, 25, 33, 108, 110, 111, 115, 129, 136, 155, 158, 172]

In [ ]:
check_strategy(check3,3)
print(len(list_strategy(3)))
list_strategy(3)

11


[30, 42, 67, 77, 79, 93, 113, 159, 174, 179, 180]

In [ ]:
check_strategy(check4,4)
print(len(list_strategy(4)))
list_strategy(4)

13


[14, 22, 23, 38, 57, 64, 85, 96, 97, 105, 148, 167, 197]

In [ ]:
check_strategy(check5,5)
print(len(list_strategy(5)))
list_strategy(5)

1


[118]

In [ ]:
# Function to calculate moving average
def calculate_moving_average(actions, alpha):
    weighted_avg = []
    actions = actions.reset_index()
    for i in range(len(actions)):
        total_weighted_sum = 0
        sum_of_weights = 0
        for j in range(i + 1):
            total_weighted_sum += actions.iloc[i - j, 1] * (alpha ** j)
            sum_of_weights += alpha ** j
        weighted_avg.append(total_weighted_sum / sum_of_weights)
    return weighted_avg


def player_plot(player_id, alpha=0.9):
    # Assuming alpha value for the moving average

    # Player ID you're interested in (p1_id or p2_id)

    # Selecting games where the player has played
    selected_games = df[(df['p1_id'] == player_id) | (df['p2_id'] == player_id)]

    # Resetting index for easy iteration
    selected_games = selected_games.reset_index(drop=True)

    # Plotting
    fig, axs = plt.subplots()

    # Dictionary to store colors for each game_id
    color_dict = {}
    x_coords = []
    y_coords = []
    for idx, game_id in enumerate(selected_games['game_id'].unique()):
        game_data = selected_games[selected_games['game_id'] == game_id]
        game_data = game_data.reset_index()
        if game_data['p2_id'].iloc[0] == player_id:
            # Calculate moving average for p1's actions
            game_data['MA1'] = calculate_moving_average(game_data['p1_action'], alpha)
            # Shift p2's actions
            df_shifted_index = game_data['p2_action'].shift(-1)
            axs.scatter(game_data['MA1'], df_shifted_index, label=f'Game {game_id}', color=f'C{idx}')
            x_coords += game_data['MA1'].tolist()
            y_coords += df_shifted_index.tolist()
        else:
            # Calculate moving average for p2's actions
            game_data['MA2'] = calculate_moving_average(game_data['p2_action'], alpha)
            # Shift p1's actions
            df_shifted_index = game_data['p1_action'].shift(-1)
            axs.scatter(game_data['MA2'], df_shifted_index, label=f'Game {game_id}', color=f'C{idx}')
            x_coords += game_data['MA2'].tolist()
            y_coords += df_shifted_index.tolist()

        color_dict[game_id] = f'C{idx}'

    axs.set_xlabel('MA')
    axs.set_ylabel('p_action (Shifted Index)')
    axs.set_title('MA vs. Shifted p_action')

    df_plot = pd.DataFrame({'MA': x_coords, 'p_action': y_coords})
    sns.catplot(kind='violin', data=df_plot, hue="p_action", x="MA", split=True)
    plt.show()


In [ ]:
len(unclassified_players())

149

A large number of people are still unclassified